# Financial Document Analysis with Retrieval-Augmented Generation (RAG)

This Jupyter Notebook demonstrates a complete workflow for building a Q&A system from financial reports using a Retrieval-Augmented Generation (RAG) architecture. We will process Kyndryl's annual reports, build a sophisticated retrieval system, and re-rank the results for higher accuracy.

### Part 1: Data Collection & Preprocessing

**1.1: Obtain and Ingest Financial Statements**

First, we'll install the necessary libraries for our project. Then, we will download the annual reports for the last two fiscal years from the provided URLs and store them locally. This step ensures we have the raw data ready for processing.

In [49]:
# !pip install PyMuPDF sentence-transformers faiss-cpu rank_bm25 transformers pandas -q
import logging
import time
from pathlib import Path
import pandas as pd
from docling.document_converter import DocumentConverter
_log = logging.getLogger(__name__)

# Download financial reports for KYNDRYL HOLDINGS, INC.

In [50]:
import requests
import os

# Create a directory to store the PDFs
if not os.path.exists("annual_reports"):
    os.makedirs("annual_reports")

pdf_urls = [
    "https://investors.kyndryl.com/static-files/0205a5a1-2f59-4ab7-b892-58615604423a",  # 2024 Annual Report
    "https://investors.kyndryl.com/static-files/1488970a-672b-4caa-ad23-00c77e2b2434",  # 2023 Annual Report
]
pdf_paths = []
for url in pdf_urls:
    file_name = os.path.join("annual_reports", url.split("/")[-1]) + ".pdf"
    if not os.path.exists(file_name):
        print(f"Downloading {file_name}...")
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(file_name, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    else:
        print(f"{file_name} already exists.")
    pdf_paths.append(file_name)

annual_reports/0205a5a1-2f59-4ab7-b892-58615604423a.pdf already exists.
annual_reports/1488970a-672b-4caa-ad23-00c77e2b2434.pdf already exists.


**1.2: Convert Documents to Plain Text and Clean**

We will use the `Docling` library to parse the downloaded PDF files and extract raw text. We will then apply a basic cleaning function to remove common artifacts like headers, footers, and extra whitespace, which are irrelevant for our Q&A task.

The 2024 financial report contains the data for both the year 2024 and 2023.

In [51]:
def parse_pdf(pdf_path, start, end):
    print(f"Parsing {pdf_path} from page {start} to {end}")
    res = []
    logging.basicConfig(level=logging.INFO)

    input_doc_path = pdf_path
    output_dir = Path("scratch")

    doc_converter = DocumentConverter()

    start_time = time.time()

    conv_res = doc_converter.convert(input_doc_path, page_range=(start, end))

    output_dir.mkdir(parents=True, exist_ok=True)

    doc_filename = conv_res.input.file.stem

    # Export tables
    for table_ix, table in enumerate(conv_res.document.tables):
        table_df: pd.DataFrame = table.export_to_dataframe()
        print(f"## Table {table_ix}")
        res.append(table_df)
        print(table_df.to_markdown())

        # Save the table as csv
        element_csv_filename = output_dir / f"{doc_filename}-table-{table_ix + 1}.csv"
        _log.info(f"Saving CSV table to {element_csv_filename}")
        table_df.to_csv(element_csv_filename)

    end_time = time.time() - start_time

    _log.info(f"Document converted and tables exported in {end_time:.2f} seconds.")
    return res

# let's parse pdf from the pages 57 to 59. The pages that contain financial tables.
# The 2024 financial report contains the data for both the year 2024 and 2023.
raw_documents = parse_pdf(pdf_paths[0], 57, 59)


INFO:docling.datamodel.document:detected formats: [<InputFormat.PDF: 'pdf'>]
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for StandardPdfPipeline with options hash 5a43216f093a7c32c3d8090bbb471faa
INFO:docling.utils.accelerator_utils:Accelerator device: 'mps'


Parsing annual_reports/0205a5a1-2f59-4ab7-b892-58615604423a.pdf from page 57 to 59


INFO:docling.utils.accelerator_utils:Accelerator device: 'mps'
INFO:docling.utils.accelerator_utils:Accelerator device: 'mps'
INFO:docling.pipeline.base_pipeline:Processing document 0205a5a1-2f59-4ab7-b892-58615604423a.pdf
INFO:docling.document_converter:Finished converting document 0205a5a1-2f59-4ab7-b892-58615604423a.pdf in 23.28 sec.
INFO:__main__:Saving CSV table to scratch/0205a5a1-2f59-4ab7-b892-58615604423a-table-1.csv
INFO:__main__:Saving CSV table to scratch/0205a5a1-2f59-4ab7-b892-58615604423a-table-2.csv
INFO:__main__:Saving CSV table to scratch/0205a5a1-2f59-4ab7-b892-58615604423a-table-3.csv
INFO:__main__:Document converted and tables exported in 23.31 seconds.


## Table 0
|    |                                                                      | Notes   | Year Ended.2024   | March 31,.2023   | Three Months Ended March 31,.2022   | Year Ended December 31,.2021   |
|---:|:---------------------------------------------------------------------|:--------|:------------------|:-----------------|:------------------------------------|:-------------------------------|
|  0 | Revenues * . . . . . . . . . . . . . . . . . . . . . . . . . . . . . | 3       | $ 16,052          | $ 17,026         | $ 4,431                             | $ 18,657                       |
|  1 | Cost of services ** . . . . . . . . . . . . . . . . . . . . . . .    | 3       | $ 13,189          | $ 14,498         | $ 3,824                             | $ 16,550                       |
|  2 | Selling, general and administrative expenses . .                     |         | 2,773             | 2,914            | 690                                 | 2,776                          |

### Clean text and Segment reports into logical sections

The above parsed document has financial data for 2024, 2023, 2022 and 2021. Let's clean up and extract for 2024 and 2023.


#### Process INCOME statement
1. KYNDRYL HOLDINGS, INC. CONSOLIDATED INCOME STATEMENT
2. KYNDRYL HOLDINGS, INC. CONSOLIDATED STATEMENT OF COMPREHENSIVE INCOME (LOSS)

In [ ]:
from itertools import chain

qa_documents = {}

questions_2023 = []
questions_2024 = []

# iterate over income statement tables and create simple Q&A pairs
for row in chain(raw_documents[0].itertuples(), raw_documents[1].itertuples()):
    if not row[3] and not row[4]:
        continue
    question_suffix = "?"
    question_prefix = f'{row[1].strip(".* :").rstrip()} of'
    answer_suffix = "." if "per share" in question_prefix.lower() else " million."

    question = question_prefix + " 2024" + question_suffix
    answer = row[3]
    answer = "Not applicable" if answer == "-" else (answer + answer_suffix)
    questions_2024.append((question, answer))

    question = question_prefix + " 2023" + question_suffix
    answer = row[4]
    answer = "Not applicable" if answer == "-" else (answer + answer_suffix)
    questions_2023.append((question, answer))

print("Sample Questions and Answers")
print(questions_2024[:2])
print(questions_2023[1:3])

qa_documents['Income Statement 2024'] = questions_2024
qa_documents['Income Statement 2023'] = questions_2023

Sample Questions and Answers
[('Revenues of 2024?', '$ 16,052 million.'), ('Cost of services of 2024?', '$ 13,189 million.')]
[('Cost of services of 2023?', '$ 14,498 million.'), ('Selling, general and administrative expenses of 2023?', '2,914 million.')]


#### Process KYNDRYL HOLDINGS, INC. CONSOLIDATED BALANCE SHEET
(In millions, except per share amounts)

In [53]:
questions_2023 = []
questions_2024 = []
common_prefix = ""
skip = False

# iterate over the balance sheet table and create simple Q&A pairs
for row in raw_documents[2].itertuples():
    question_suffix = "?"
    question_prefix = f'{row[1].rstrip(".* :").rstrip()}'
    if question_prefix.endswith("Assets") and not row[3]:
        common_prefix = " assets"
    elif question_prefix.endswith("Liabilities") and not row[3]:
        common_prefix = " liabilities"
    elif question_prefix.endswith("Equity") and not row[3]:
        break
    if not row[3] and not row[4]:
        continue

    if not question_prefix:
        skip = True
        continue
    elif skip:
        skip = False
        continue

    answer_suffix = "." if "per share" in question_prefix.lower() else " million."

    question = (
        (
            question_prefix
            if question_prefix.endswith(common_prefix)
            else f"{question_prefix}{common_prefix}"
        )
        + " of 2024"
        + question_suffix
    )
    answer = row[3] + answer_suffix
    questions_2024.append((question, answer))
    question = (
        (
            question_prefix
            if question_prefix.endswith(common_prefix)
            else f"{question_prefix}{common_prefix}"
        )
        + " of 2023"
        + question_suffix
    )
    answer = row[4] + answer_suffix
    questions_2023.append((question, answer))

print("Sample Questions and Answers")
print(questions_2024[-2:])
print(questions_2023[-1:])

print(f"Total 2024 Q&A pairs: {len(questions_2024)}")
print(f"Total 2023 Q&A pairs: {len(questions_2023)}")

qa_documents['Balance Sheet 2024'] = questions_2024
qa_documents['Balance Sheet 2023'] = questions_2023

Sample Questions and Answers
[('Other noncurrent liabilities of 2024?', '332 million.'), ('Total liabilities of 2024?', '$ 9,468 million.')]
[('Total liabilities of 2023?', '$ 10,002 million.')]
Total 2024 Q&A pairs: 26
Total 2023 Q&A pairs: 26


#### Q/A Pairs

In [54]:
for doc, questions in qa_documents.items():
    print(f"\nDocument: {doc}")
    for q, a in questions:
        print(f"Q: {q}\nA: {a}\n")


Document: Income Statement 2024
Q: Revenues of 2024?
A: $ 16,052 million.

Q: Cost of services of 2024?
A: $ 13,189 million.

Q: Selling, general and administrative expenses of 2024?
A: 2,773 million.

Q: Workforce rebalancing charges of 2024?
A: 138 million.

Q: Transaction-related costs (benefits) of 2024?
A: (46) million.

Q: Impairment expense of 2024?
A: Not applicable

Q: Interest expense of 2024?
A: 122 million.

Q: Other expense of 2024?
A: 45 million.

Q: Total costs and expenses of 2024?
A: $ 16,221 million.

Q: Income (loss) before income taxes of 2024?
A: $ (168) million.

Q: Provision for income taxes of 2024?
A: $ 172 million.

Q: Net income (loss) of 2024?
A: $ (340) million.

Q: Basic earnings (loss) per share of 2024?
A: $ (1.48).

Q: Diluted earnings (loss) per share of 2024?
A: (1.48).

Q: Weighted-average basic shares outstanding of 2024?
A: 229.2 million.

Q: Weighted-average diluted shares outstanding of 2024?
A: 229.2 million.

Q: Net income (loss) of 2024?
A: $

### Part 2: Retrieval-Augmented Generation (RAG) System Implementation

**2.1 Data Processing: Chunking**

To prepare the text for retrieval, we'll split it into smaller, manageable chunks. This allows the model to find more specific and relevant passages. We will create two sets of chunks with different sizes (100 and 400 tokens) to analyze the impact of chunk size on retrieval performance. Each chunk will be assigned a unique ID and metadata indicating its source document and chunk size.

The metadata will have info whether the chunk is courced from balance sheet or income statement segments.

In [55]:
from transformers import AutoTokenizer

model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def chunk_text(qa_list, chunk_size, source_key):
    chunks = []
    for idx, (q, a) in enumerate(qa_list):
        text_block = f"Q: {q}\nA: {a}"
        tokens = tokenizer.encode(text_block)
        for i in range(0, len(tokens), chunk_size):
            chunk_tokens = tokens[i:i + chunk_size]
            chunk_str = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
            chunks.append({
                "id": f"{source_key}_{idx}_size_{chunk_size}_chunk_{len(chunks)}",
                "text": chunk_str,
                "metadata": {
                    "segment": source_key,
                    "qa_index": idx,
                    "chunk_size": chunk_size
                }
            })
    return chunks

chunk_sizes = [100, 400]
all_chunks = []

for size in chunk_sizes:
    for key in qa_documents:
        all_chunks.extend(chunk_text(qa_documents[key], size, key))

print(f"Total number of chunks created: {len(all_chunks)}")
print("Sample chunk:", all_chunks[0])

Total number of chunks created: 236
Sample chunk: {'id': 'Income Statement 2024_0_size_100_chunk_0', 'text': 'q : revenues of 2024? a : $ 16, 052 million.', 'metadata': {'segment': 'Income Statement 2024', 'qa_index': 0, 'chunk_size': 100}}


**2.2 Embedding & Indexing**

Next, we will convert the text chunks into numerical vectors (embeddings) using the `all-MiniLM-L6-v2` model. These embeddings capture the semantic meaning of the text.

We will build two types of indexes:
1.  **Dense Vector Store (FAISS):** For fast semantic similarity search.
2.  **Sparse Index (BM25):** For efficient keyword-based retrieval.

In [56]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from rank_bm25 import BM25Okapi

# 1. Embed Chunks
embedding_model = SentenceTransformer(model_name)
chunk_texts = [chunk['text'] for chunk in all_chunks]
embeddings = embedding_model.encode(chunk_texts, convert_to_numpy=True, show_progress_bar=True)

# 2. Build Dense Vector Store (FAISS)
embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)
faiss_index.add(np.array(embeddings, dtype=np.float32))
print(f"FAISS index built with {faiss_index.ntotal} vectors.")

# 3. Build Sparse Index (BM25)
tokenized_corpus = [doc.lower().split(" ") for doc in chunk_texts]
bm25 = BM25Okapi(tokenized_corpus)
print("BM25 index built.")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
Batches: 100%|██████████| 8/8 [00:00<00:00,  8.70it/s]

FAISS index built with 236 vectors.
BM25 index built.


**2.3 Hybrid Retrieval Pipeline**

Our retrieval pipeline will combine the strengths of both dense and sparse methods. For a given query, we will:
1. Preprocess the query.
2. Retrieve the top N chunks from FAISS based on vector similarity.
3. Retrieve the top N chunks from BM25 based on keyword matching.
4. Combine the results using a simple union to create a comprehensive list of candidate chunks.

In [57]:
def preprocess_query(query):
    # Normalize
    query = query.lower()
    
    # All the queries are about Kyndryl. So, let's remove that, it might bias the retriever
    # Remove redundant keywords
    stopwords = ["kyndryl", "IBM", "company", "inc", "inc.", "corporation", "corp", "corp.", "ltd", "ltd.", "plc", "the"]
    for w in stopwords:
        query = query.replace(w.lower(), "")
    return query.strip()

def hybrid_retrieve(query, top_n=5):
    # 1. Preprocess query
    clean_query = preprocess_query(query)
    
    # 2. Dense Retrieval (FAISS)
    query_embedding = embedding_model.encode([clean_query])
    _, dense_indices = faiss_index.search(np.array(query_embedding, dtype=np.float32), top_n)
    dense_results = [all_chunks[i] for i in dense_indices[0]]
    
    # 3. Sparse Retrieval (BM25)
    tokenized_query = clean_query.split(" ")
    bm25_scores = bm25.get_scores(tokenized_query)
    sparse_indices = np.argsort(bm25_scores)[::-1][:top_n]
    sparse_results = [all_chunks[i] for i in sparse_indices]
    
    # 4. Combine results
    combined_results_dict = {chunk['id']: chunk for chunk in dense_results + sparse_results}
    
    print(f"Retrieved {len(dense_results)} chunks from dense search.")
    print(f"Retrieved {len(sparse_results)} chunks from sparse search.")
    print(f"Combined to {len(combined_results_dict)} unique chunks.")
    
    return list(combined_results_dict.values())

In [58]:
# Example Usage
test_query = "What was the company's revenue in 2024?"
retrieved_chunks = hybrid_retrieve(test_query)

print(f"\n--- Top Retrieved Chunk for '{test_query}' ---")
print(retrieved_chunks[0]['text'])

Batches: 100%|██████████| 1/1 [00:00<00:00, 62.71it/s]

Retrieved 5 chunks from dense search.
Retrieved 5 chunks from sparse search.
Combined to 10 unique chunks.

--- Top Retrieved Chunk for 'What was the company's revenue in 2024?' ---
q : revenues of 2024? a : $ 16, 052 million.


**2.4 Advanced RAG Technique: Cross-Encoder Re-ranking**

The initial retrieval might return chunks that are only partially relevant. To refine our results, we'll use a Cross-Encoder model. Unlike the embedding model which computes vectors independently, a Cross-Encoder takes both the query and a candidate chunk as input to produce a more accurate relevance score. We will use this to re-rank the top chunks retrieved from our hybrid pipeline.

In [59]:
from sentence_transformers.cross_encoder import CrossEncoder

# Load a cross-encoder model
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

def rerank_with_cross_encoder(query, chunks):
    # Create pairs of [query, chunk_text] for scoring
    query_chunk_pairs = [[query, chunk['text']] for chunk in chunks]
    
    # Get scores from the cross-encoder
    scores = cross_encoder.predict(query_chunk_pairs, show_progress_bar=False)
    
    # Add scores to chunks and sort
    for i in range(len(chunks)):
        chunks[i]['relevance_score'] = scores[i]
        
    reranked_chunks = sorted(chunks, key=lambda x: x['relevance_score'], reverse=True)
    return reranked_chunks


def advanced_retrieve(query):
    retrieved_chunks = hybrid_retrieve(query)
    reranked_results = rerank_with_cross_encoder(query, retrieved_chunks)
    return reranked_results

INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: mps


In [60]:
# Example Usage with the same test query
test_query = "What was the company's revenue in 2024?"
reranked_results = advanced_retrieve(test_query)
print(f"\n--- Top Re-ranked Chunk for '{test_query}' ---")
print(f"Relevance Score: {reranked_results[0]['relevance_score']:.4f}")
print(reranked_results[0]['text'])

Batches: 100%|██████████| 1/1 [00:00<00:00, 12.71it/s]

Retrieved 5 chunks from dense search.
Retrieved 5 chunks from sparse search.
Combined to 10 unique chunks.

--- Top Re-ranked Chunk for 'What was the company's revenue in 2024?' ---
Relevance Score: 8.9147
q : revenues of 2024? a : $ 16, 052 million.


**2.5 Response Generation**

After retrieving and re-ranking the most relevant document chunks, the final step is to generate a coherent, human-readable answer. We will use a generative language model to synthesize the information from the retrieved passages into a direct response to the user's query. 

For this demonstration, we'll use `Flan T5`, a smaller and more efficient version, which is suitable for tasks where resource constraints are a consideration. The retrieved text and the original query are combined into a carefully crafted prompt to guide the model in generating a factual answer grounded in the provided context.

In [61]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# --- Configuration ---
# Using google/flan-t5-base is a good balance for an 8GB M2 Mac.
# 'small' is faster but less accurate; 'large' may run out of memory.
model_name = "google/flan-t5-base"

# --- Model and Tokenizer Loading ---
# Load the model and tokenizer, sending the model to the specified device
try:
    gen_tokenizer = AutoTokenizer.from_pretrained(model_name)
    gen_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure you have a stable internet connection.")
    exit()

# --- Initialize Pipeline ---
# Initialize the pipeline once for efficiency, specifying the correct task and device
# For MPS, device=0 is the standard way to reference the first MPS device.
nlp_pipeline = pipeline(
    "text2text-generation",
    model=gen_model,
    tokenizer=gen_tokenizer,
)

def generate_answer(query, reranked_chunks, max_new_tokens=100):
    """
    Generates an answer using the Flan-T5 model based on the retrieved chunks.
    """
    # 1. Prepare the context from the top 3 reranked chunks
    context = "\n\n".join([chunk.get("text", "") for chunk in reranked_chunks[:3]])

    # 2. Create a prompt suitable for Flan-T5
    # This instruction-based format works well for this model family.
    prompt = f"""
Based on the following context, please answer the question.

Context:
{context}

Question:
{query}

Answer:
"""

    # 3. Generate the answer using the pre-initialized pipeline
    # The pipeline handles tokenization, truncation, and moving data to the MPS device.
    generated_output = nlp_pipeline(
        prompt,
        max_length=512,  # Set a max_length for the input+output
        max_new_tokens=max_new_tokens,  # Control the max length of the generated answer
        num_return_sequences=1,
    )

    # 4. Extract and clean the answer text
    # The output from a 'text2text-generation' pipeline is cleaner.
    answer = generated_output[0]["generated_text"].strip()

    return answer

Device set to use mps:0


In [72]:
def query_finance_system_rag(query, debug = False):
    reranked_results = advanced_retrieve(query)
    
    if debug:
        for r in reranked_results:
            print('-----')
            print(r['text'])

    # Example Usage
    final_answer = generate_answer(query, reranked_results)
    return final_answer

test_query = "What is fiscal year 2024 revenue?"
print(f"Question: {test_query}")
final_answer = query_finance_system_rag(test_query)
print(f"\nGenerated Answer:\n{final_answer}")

Question: What is fiscal year 2024 revenue?


Batches: 100%|██████████| 1/1 [00:04<00:00,  4.55s/it]


Retrieved 5 chunks from dense search.
Retrieved 5 chunks from sparse search.
Combined to 10 unique chunks.


Both `max_new_tokens` (=100) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Generated Answer:
$ 16, 052 million


**2.6 Guardrail Implementation**

To ensure the reliability and safety of our RAG system, we need to implement guardrails. These are checks and balances that prevent the system from processing inappropriate queries or generating harmful, irrelevant, or factually incorrect answers. We will implement two basic guardrails:

1.  **Input Guardrail:** A simple filter to block off-topic or nonsensical questions.
2.  **Output Guardrail:** A check to ensure the generated answer is grounded in the retrieved context and not a hallucination.

In [73]:
# --- Input Guardrail ---

def validate_query(query):
    """
    Validates the input query to filter out irrelevant or harmful inputs.
    """
    query = query.lower()
    
    # Check for minimum length
    if len(query.split()) < 3:
        return False, "Query is too short. Please ask a more specific question."
        
    # Basic check for off-topic keywords
    off_topic_keywords = ['recipe', 'poem', 'joke', 'movie', 'celebrity']
    if any(keyword in query for keyword in off_topic_keywords):
        return False, "Query is off-topic. This system is for financial document analysis."

    return True, "Query is valid."

# --- Output Guardrail ---

def validate_output(answer, retrieved_chunks):
    """
    Validates the generated output to flag potential hallucinations.
    This is a simple check based on keyword overlap.
    """
    context = " ".join([chunk['text'].lower() for chunk in retrieved_chunks])
    answer_tokens = set(answer.lower().split())
    context_tokens = set(context.split())
    
    if not answer_tokens:
        return False, "Generated answer is empty."

    # Calculate the percentage of answer tokens that are present in the context
    overlap = answer_tokens.intersection(context_tokens)
    overlap_ratio = len(overlap) / len(answer_tokens)
    
    # If overlap is less than a certain threshold (e.g., 30%), flag it as potentially ungrounded.
    if overlap_ratio < 0.3:
        return False, f"Potential hallucination detected. Answer may not be grounded in context (Overlap: {overlap_ratio:.2%})."
        
    return True, f"Answer appears grounded in context (Overlap: {overlap_ratio:.2%})."


# --- Example Usage of Guardrails ---

# 1. Input Guardrail Examples
print("--- Input Guardrail Tests ---")
valid_query = "What were the total assets of Kyndryl as of March 31, 2024?"
invalid_query_short = "revenue?"
invalid_query_topic = "Can you give me a recipe for a cake?"

is_valid, reason = validate_query(valid_query)
print(f"Query: '{valid_query}' -> Valid: {is_valid}, Reason: {reason}")

is_valid, reason = validate_query(invalid_query_short)
print(f"Query: '{invalid_query_short}' -> Valid: {is_valid}, Reason: {reason}")

is_valid, reason = validate_query(invalid_query_topic)
print(f"Query: '{invalid_query_topic}' -> Valid: {is_valid}, Reason: {reason}")

print("\n" + "="*50 + "\n")

# 2. Output Guardrail Example
print("--- Output Guardrail Tests ---")
# Use the previously generated answer
is_grounded, reason = validate_output(final_answer, reranked_results)
print(f"Answer: '{final_answer[:100]}...' -> Grounded: {is_grounded}, Reason: {reason}")

# Example of a potentially hallucinated answer
hallucinated_answer = "Kyndryl announced a partnership with SpaceX to build datacenters on Mars."
is_grounded, reason = validate_output(hallucinated_answer, reranked_results)
print(f"Answer: '{hallucinated_answer}' -> Grounded: {is_grounded}, Reason: {reason}")

--- Input Guardrail Tests ---
Query: 'What were the total assets of Kyndryl as of March 31, 2024?' -> Valid: True, Reason: Query is valid.
Query: 'revenue?' -> Valid: False, Reason: Query is too short. Please ask a more specific question.
Query: 'Can you give me a recipe for a cake?' -> Valid: False, Reason: Query is off-topic. This system is for financial document analysis.


--- Output Guardrail Tests ---
Answer: '$ 16, 052 million...' -> Grounded: True, Reason: Answer appears grounded in context (Overlap: 75.00%).
Answer: 'Kyndryl announced a partnership with SpaceX to build datacenters on Mars.' -> Grounded: False, Reason: Potential hallucination detected. Answer may not be grounded in context (Overlap: 9.09%).


### 3. Fine-Tuned Model System Implementation
